# Notebook: 03_analise_gold

O objetivo deste notebook é consolidar os dados tratados da camada Silver e gerar indicadores relevantes sobre a criminalidade no Estado de São Paulo, com foco em frequência de crimes, comparações temporais, distribuição geográfica e taxa de criminalidade ajustada por população.

#### **1. Definição do Problema**

- O projeto visa analisar a **frequência, geolocalização e tipos de crimes cometidos no Estado de São Paulo, ao longo de três anos**.
- O **usuário final** será a **população interessada na criminalidade** ou a **Polícia Civil do Estado de São Paulo**.


#### **2. Fonte de Dados**

- **Base de dados principal:** Boletins de ocorrência da **SSP-SP** (Secretaria de Segurança Pública de SP), disponibilizados diariamente.
- **Dados complementares:** Base de dados demográficos do estado de São Paulo do **IBGE**.


#### **3. Formato dos Dados**

- Os dados estão **em formato CSV**.
- Estruturados, prontos para processamento.

#### **4. Histórico e Frequência dos Dados**

- O dataset cobre um **histórico amplo**, contendo informações de **janeiro 2022 até dezembro de 2024**.
- A análise será baseada em **dados mensais**.

#### Perguntas Objetivo
O trabalho tem a função de responder as perguntas abaixo, separadas em duas partes.

Perguntas relacionadas aos registros em todo o estado de São Paulo:

- Quais as 5 categorias de crime mais registrados no Estado?
- Esses 5 permaneceram os mesmos em 2022, 2023 e 2024?
- Houve sazonalidade? Com naturezas mais registradas ou menos registradas em determinadas épocas do ano? Ou eventos de aumento ou redução drásticas de determinados registros?
- Qual a distribuição geográfica das categorias de crime dentro do estado de São Paulo?

Perguntas relacionadas à distribuição dos tipos ou categorias de crime nos diferentes municípios do estado:

- Qual o percentual de registro de cada tipo de crime nos anos de 2022, 2023 e 2024 em cada município de São Paulo?
- Qual a distribuição geográfica das categorias de crime dentro de cada município?
- Qual a taxa de registro a cada 100 mil habitantes das 3 maiores categorias de crime de cada município?

## 1. Setup Inicial

In [0]:
# imports essenciais do pyskpark - separado para o Databricks não esquecer
from pyspark.sql import SparkSession

In [0]:
from pyspark.sql.functions import desc, sum, lit, round, when

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, count

In [0]:
from pyspark.sql.types import IntegerType, FloatType
from functools import reduce
from pyspark.sql import DataFrame

In [0]:
%python
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

In [0]:
%matplotlib inline

In [0]:
# definindo estilo para os próximos gráficos
sns.set_theme(
    style="whitegrid",
    rc={
        "axes.grid": True,
        "grid.color": "#e6e6e6",  # cinza bem claro
        "grid.linewidth": 0.7
    }
)

In [0]:
# criando a sessão
spark = SparkSession.builder.getOrCreate()

## 2. Importando os dados Silver

In [0]:
# lendo os dados de criminalidade ano a ano
df_2022 = spark.read.parquet("/FileStore/mvp_criminalidade_sp/silver/ano=2022").withColumn("ano", lit(2022))
df_2023 = spark.read.parquet("/FileStore/mvp_criminalidade_sp/silver/ano=2023").withColumn("ano", lit(2023))
df_2024 = spark.read.parquet("/FileStore/mvp_criminalidade_sp/silver/ano=2024").withColumn("ano", lit(2024))

# unindo os anos
df_silver = reduce(DataFrame.unionByName, [df_2022, df_2023, df_2024])

df_silver.printSchema()

In [0]:
# lendo os arquivos do ibge
df_ibge = spark.read.parquet("/FileStore/mvp_criminalidade_sp/silver/ibge")

df_ibge.printSchema()

## 3. Preparando os dados para análise
Criação de colunas e união dos databases

### 3.1 Calculando a população para 2023
Vamos criar uma coluna nova importante, a coluna `taxa_100_mil`, que é correspondente à taxa de registros de crimes a cada 100 mil pessoas, para isso precisamos estimar a população do ano de 2023. Para essa estimativa vamos calcular a média entre a população do censo de 2022 e a população estimada de 2024 (talvez não seja a melhor abordagem, mas por enquanto vamos com ela).

In [0]:
# calculando a média
df_ibge = (
    df_ibge
    .withColumn("pop_2023", round((col("pop_censo_22") + col("pop_estimada_24")) / 2))
)

# checando o resultado
df_ibge.select("municipio", "pop_censo_22", "pop_estimada_24", "pop_2023").show(10, truncate=False)

### 3.3 Criando as bases definitivas `df_bo` e `df_municipios` (com as taxas de registro de criminalidade)
As duas bases tratam de informações completamente diferentes, por isso vamos unificar todos os dados que tratam do boletim de ocorrência na df_bos e a df_municipios concentra todas as informações por município. Nela vamos concentrar o cálculo da taxa de registro de crimes nas três divisões que temos:

- `natureza` - que é literalmente a Natureza Apurada do boletim, a informação mais específica
- `categoria` - que é baseado no agrupamento que eu criei, com crimes "similares"
- `tipo_crime` - que é de acordo com o agrupamento do Código Penal Brasileiro

In [0]:
df_qtd_bos = (
    df_silver
    .groupBy("ano", "municipio")
    .agg(count("*").alias("qtd_bos"))
)

In [0]:
df_pop = df_ibge.select(
    "municipio",
    "codigo_ibge",
    col("pop_censo_22").alias("populacao_22"),
    col("pop_2023").alias("populacao_23"),
    col("pop_estimada_24").alias("populacao_24")
)

In [0]:
df_municipio_temp = df_qtd_bos.join(
    df_pop,
    on=["municipio"],
    how="left"
)

In [0]:
df_municipio_temp = df_municipio_temp.withColumn(
    "taxa_100k",
    when(col("ano") == 2022, round((col("qtd_bos") / col("populacao_22")) * 100000, 2))
    .when(col("ano") == 2023, round((col("qtd_bos") / col("populacao_23")) * 100000, 2))
    .when(col("ano") == 2024, round((col("qtd_bos") / col("populacao_24")) * 100000, 2))
)

In [0]:
df_taxa_22 = df_municipio_temp.filter(col("ano") == 2022).select(
    "municipio", "codigo_ibge", col("taxa_100k").alias("taxa_100k_22")
)

df_taxa_23 = df_municipio_temp.filter(col("ano") == 2023).select(
    "municipio", "codigo_ibge", col("taxa_100k").alias("taxa_100k_23")
)

df_taxa_24 = df_municipio_temp.filter(col("ano") == 2024).select(
    "municipio", "codigo_ibge", col("taxa_100k").alias("taxa_100k_24")
)

In [0]:
df_municipio = (
    df_taxa_22
    .join(df_taxa_23, on=["municipio", "codigo_ibge"], how="outer")
    .join(df_taxa_24, on=["municipio", "codigo_ibge"], how="outer")
)

In [0]:
df_municipio.printSchema()

In [0]:
# tirei umas colunas sem querer, vamos juntar de novo
df_municipio = df_municipio.join(
    df_ibge.select(
        "municipio",
        "codigo_ibge",
        col("pop_censo_22").alias("populacao_22"),
        col("pop_2023").alias("populacao_23"),
        col("pop_estimada_24").alias("populacao_24")
    ),
    on=["municipio", "codigo_ibge"],
    how="left"
)

In [0]:
df_municipio.printSchema()

Pronto, agora temos as duas bases completinhas para responder todas as perguntas. 

## 4. Respondendo as Perguntas Objetivo
Vamos uma por uma, por mais que algumas vão ser respondidas fora daqui, com outro recurso.

### 4.1 Quais as 5 categorias de crime mais registradas no estado de São Paulo?
Essa pergunta é baseada na subdivisão `categoria`, por isso vamos calcular com ela.

In [0]:
# calculando as top 5 categorias do estado inteiro.
top_categorias = (
    df_silver
    .groupBy("categoria")
    .agg(count("*").alias("total_ocorrencias"))
    .orderBy(desc("total_ocorrencias"))
    .limit(5)
)

top_categorias.show(truncate=False)

Como podemos ver, as 5 maiores categorias são `furto`, `roubo`, `lesao corporal`, `entorpecentes` e `estupro`. Sendo a diferença entre a primeira e a segunda de **1,9 milhão** de registros para **776 mil**, quase **3 vezes maior**. Já a distância entre a **primeira e a quinta categoria** é de **1,9 milhão** para **42 mil**, **45 vezes maior**.
A próxima pergunta vai trazer clareza se essa é uma tendência que se repete ao longo dos anos.

### 4.2 Esses 5 permaneceram os mesmos em 2022, 2023 e 2024?
Vamos gerar um gráfico de barra por ano lado a lado para comparar o top 5.

In [0]:
df_top5_22 = df_top5_pd[df_top5_pd["ano"] == 2022].sort_values("total_mil", ascending=False)
df_top5_23 = df_top5_pd[df_top5_pd["ano"] == 2023].sort_values("total_mil", ascending=False)
df_top5_24 = df_top5_pd[df_top5_pd["ano"] == 2024].sort_values("total_mil", ascending=False)

In [0]:
# criando a paleta das categorias
custom_palette = {
    "furto": "#A6CEE3",         # azul pastel
    "roubo": "#FDBF6F",         # laranja pastel
    "lesao corporal": "#B2DF8A",# verde pastel
    "entorpecentes": "#FB9A99", # vermelho rosado pastel
    "estupro": "#CAB2D6"        # lilás pastel
}

In [0]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5), constrained_layout=True)

anos = [2022, 2023, 2024]
dfs = [df_top5_22, df_top5_23, df_top5_24]

for ax, ano, df in zip(axes, anos, dfs):
    sns.barplot(
        data=df,
        x="total_mil",
        y="categoria",
        palette=custom_palette,
        orient="h",
        ax=ax
    )
    
    for patch in ax.patches:
        width = patch.get_width()
        y = patch.get_y() + patch.get_height() / 2
        ax.text(
            width + 3,
            y,
            f"{width:.1f} mil",
            va="center",
            ha="left",
            fontsize=10,
            color="black"
        )
    
    ax.set_title(f"{ano}", fontsize=13)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xlim(0, 750)
    
    # Remove bordas laterais
    for spine in ax.spines.values():
        spine.set_visible(False)

# Título geral
fig.suptitle(
    "Top 5 Categorias de Crimes mais registrados no estado de São Paulo por ano",
    fontsize=15,
    fontweight="bold",
    y=1.1
)

plt.show()

Podemos observar no gráfico que o ano de 2024 teve uma mudança no ranking, com a categoria de Lesão Corporal ultrapassando o Roubo. Ela já havia crescido de 2022 para 2023, como Lesão Corporal contém crimes de natureza bem violenta, podemos criar a hipótese que a violência no estado de São Paulo pode ter aumentado ou, de maneira mais realista, que essa categoria de crime violento foi mais reportada.

Agora vamos ver as flutuações no mês.

### 4.3 Há sazonalidade nos registros? Com naturezas mais registradas ou menos registradas em determinadas épocas do ano? Ou eventos de aumento ou redução drásticas de determinados registros?
Vamos plotar alguns gráficos para ver o comportamento ao longo do tempo.


In [0]:
# agrupando por mes e ano
df_mensal = (
    df_silver
    .groupBy("ano", "mes")
    .agg(count("*").alias("qtd_bos"))
    .orderBy("ano", "mes")
)

In [0]:
# convertendo para pandas
df_mensal_pd = df_mensal.toPandas().sort_values(["ano", "mes"])

In [0]:
# montando o gráfico de linhas
plt.figure(figsize=(10, 6))

sns.lineplot(
    data=df_mensal_pd,
    x="mes",
    y="qtd_bos",
    hue="ano",
    palette="pastel",
    marker="o"
)

plt.title("Ocorrências por Mês no Estado de SP (2022–2024)", fontsize=13, fontweight="bold")
plt.xlabel("Mês")
plt.ylabel("Quantidade de BOs")
plt.xticks(range(1, 13))  # Garante meses de 1 a 12
plt.tight_layout()
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.show()

Como podemos observar, houveram tendências interessantes no volume de registro:

**Tendências entre os três anos**
- Podemos observar uma redução generalizada de registros em fevereiro;
- Em abril, julho e outubro a quantidade de registros parece quase constante ao longo dos 3 anos;
- 2023 teve mais registros mês a mês que 2022 e 2024 em todos os meses com exceção de abril e dezembro;
- Em 2022 houve uma redução drástica no registro de boletins de ocorrência no estado em fevereiro e, logo em seguida, um aumento drástico nos registros, em março;
- 2024 teve as menores quantidades nos três anos com exceção de abril e julho;

Essas grandes discrepâncias podem ser associadas a períodos de férias, atraso no registro dos crimes (no final de fevereiro pode ser o início do carnaval, mas as pessoas registram os crimes depois que voltam para casa, por exemplo) ou campanhas específicas que estimulem o registro de determinados crimes.

É sempre importante tomar muito cuidado ao analisar esses dados e tentar tirar conclusões com eles.

### 4.4 Qual a distribuição geográfica das categorias de crime dentro do estado de São Paulo?

Perguntas relacionadas à distribuição dos tipos ou categorias de crime nos diferentes municípios do estado:

- Qual o percentual de registro de cada tipo de crime nos anos de 2022, 2023 e 2024 em cada município de São Paulo?
- Qual a distribuição geográfica das categorias de crime dentro de cada município?
- Qual a taxa de registro a cada 100 mil habitantes das 3 maiores categorias de crime de cada município?

# Autoavaliação
Consegui atingir parcialmente os objetivos propostos para este trabalho. A principal dificuldade foi perceber tarde demais a complexidade do conjunto de análises que me propus a realizar. Ao tentar incorporar muitas visualizações e comparações complexas, acabei não respondendo a todas as perguntas delineadas inicialmente. A falta de foco na etapa de escopo tornou o fechamento do MVP mais desafiador.

O maior obstáculo técnico foi o uso da plataforma Databricks, especialmente no que diz respeito à manipulação de grandes volumes de dados com PySpark — uma biblioteca com a qual eu tinha menos familiaridade, em contraste com minha experiência prévia com pandas. Tive problemas com a importação de bibliotecas, organização de células e lentidão na execução, além da curva de aprendizado da própria plataforma.

Se eu tivesse mais tempo, faria uma análise aprofundada dos municípios do estado de São Paulo, criando dashboards com indicadores por município e cruzando essas informações com dados sociodemográficos do IBGE. Também separaria a cidade de São Paulo por zonas ou distritos e analisaria temporalidade, campos de texto dos boletins, e níveis de violência dos crimes.

Apesar disso, considero este trabalho um avanço significativo em relação à análise anterior que realizei apenas com dados de 6 meses da capital. A expansão para o estado inteiro me preparou para análises futuras mais complexas — que certamente pretendo realizar como parte do meu portfólio.